In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizer
import numpy as np
from torchmetrics import Accuracy, Precision, Recall, F1Score
from noisy_intents.data import DydaDA
from noisy_intents.training import train, autodetect_device
from noisy_intents.models import BERT
from noisy_intents.eval import compute_metrics

from transformers import get_linear_schedule_with_warmup

## Load data

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
train_data = DydaDA.from_hugging_face("train", tokenizer, max_len=128)
val_data = DydaDA.from_hugging_face("validation", tokenizer, max_len=128)

In [ ]:
train_loader = DataLoader(train_data, batch_size=8, num_workers=8, drop_last=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False, num_workers=8, drop_last=True)

## Load model

In [ ]:
device = autodetect_device()

In [ ]:
model = BERT(num_classes=4, dropout=0.1)
model.to(device);

Determine inverse class frequencies to re-weight the loss function:

In [ ]:
counts = np.bincount(train_data.data[:]["Label"])
freq = counts / counts.sum()
weights = torch.from_numpy((1 / freq) / (1 / freq).sum()).float().to(device)

Set up the optimizer and the scheduler:

In [ ]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 4
)

The training can be followed using tensorboard:

In [ ]:
train(
    epochs=4,
    model=model,
    loss_fn=torch.nn.CrossEntropyLoss(weight=weights),
    optimizer=optimizer,
    device=device,
    train_loader=train_loader,
    val_loader=val_loader,
    val_metrics_freq=1,
    scheduler=scheduler,
    metrics=[  # Log metrics on tensorboard during training
        Accuracy("multiclass", num_classes=4),
        Accuracy("multiclass", num_classes=4, average="macro"),
        Precision("multiclass", num_classes=4, average="macro"),
        Recall("multiclass", num_classes=4, average="macro"),
        F1Score("multiclass", num_classes=4, average="macro"),
    ],
    log_dir="./logs/bert_base_uncased",  # Tensorboard log_dir
)

In [ ]:
torch.save(model, "best_bert_finetuned.pt")

## Evaluate the model on the validation set

In [ ]:
compute_metrics(
    model,
    val_loader,
    device,
    metrics=[
        Accuracy("multiclass", num_classes=4, average="micro"),
        Accuracy("multiclass", num_classes=4, average="macro"),
        Accuracy("multiclass", num_classes=4, average=None),
        Precision("multiclass", num_classes=4, average="macro"),
        Recall("multiclass", num_classes=4, average="macro"),
        F1Score("multiclass", num_classes=4, average="macro"),
        F1Score("multiclass", num_classes=4, average=None),
    ],
)